In [8]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create a notebook file
with open('wetting_envelope_app.ipynb', 'w') as f:
    notebook_content = {
        "cells": [
            {
                "cell_type": "markdown",
                "metadata": {},
                "source": [
                    "# Wetting Envelope Interactive Applet\n",
                    "\n",
                    "This notebook provides an interactive interface for plotting wetting envelopes for materials and solvents. It allows you to:\n",
                    "- Plot wetting envelopes for materials with different surface energy components\n",
                    "- Add liquid points to the plot\n",
                    "- Adjust contact angles and other parameters\n",
                    "\n",
                    "The calculations are based on the Owens-Wendt model for surface energy."
                ]
            },
            {
                "cell_type": "code",
                "execution_count": None,
                "metadata": {},
                "source": [
                    "import numpy as np\n",
                    "import matplotlib.pyplot as plt\n",
                    "import ipywidgets as widgets\n",
                    "from IPython.display import display, clear_output\n",
                    "\n",
                    "# Function to calculate R(phi) for the wetting envelope\n",
                    "def calculate_R(phi, sigma_s_p, sigma_s_d):\n",
                    "    cos_phi = np.cos(phi)\n",
                    "    sin_phi = np.sin(phi)\n",
                    "    R = (np.sqrt(cos_phi * sigma_s_d) + np.sqrt(sin_phi * sigma_s_p)) / (cos_phi + sin_phi)\n",
                    "    return R**2\n",
                    "\n",
                    "# Function to plot the wetting envelope\n",
                    "def plot_wetting_envelope(components, thetas, legends, ax, title, correction_exp=2.0):\n",
                    "    ax.clear()\n",
                    "    plot_data = []\n",
                    "    \n",
                    "    # Plot each material's wetting envelope\n",
                    "    for i in range(len(thetas)):\n",
                    "        if len(components) > i:\n",
                    "            sigma_s_p = float(components[i][0])\n",
                    "            sigma_s_d = float(components[i][1])\n",
                    "        else:\n",
                    "            sigma_s_p = float(components[0][0])\n",
                    "            sigma_s_d = float(components[0][1])\n",
                    "            \n",
                    "        theta = float(thetas[i])\n",
                    "        \n",
                    "        # Convert theta to radians and calculate the correction factor\n",
                    "        theta_rad = np.radians(theta)\n",
                    "        correction_factor = 2 / (1 + np.cos(theta_rad))\n",
                    "        \n",
                    "        # Generate a range of phi values from 0 to 90 degrees (in radians) with 1000 points\n",
                    "        phi_values = np.linspace(0, np.pi / 2, 1000)\n",
                    "        R_values = calculate_R(phi_values, sigma_s_p, sigma_s_d)\n",
                    "        \n",
                    "        # Adjust R values by the correction factor\n",
                    "        R_values *= correction_factor**correction_exp\n",
                    "        \n",
                    "        # Convert to Cartesian coordinates where X is the dispersive component and Y is the polar component\n",
                    "        x_values = R_values * np.cos(phi_values)  # Dispersive component on X-axis\n",
                    "        y_values = R_values * np.sin(phi_values)  # Polar component on Y-axis\n",
                    "        \n",
                    "        # Store the data for export\n",
                    "        legend_label = legends[i] if i < len(legends) and legends[i] else f'θ = {theta}°'\n",
                    "        plot_data.append((x_values, y_values, legend_label))\n",
                    "        \n",
                    "        # Plotting the wetting envelope\n",
                    "        ax.plot(x_values, y_values, label=legend_label)\n",
                    "    \n",
                    "    # Plot liquid points\n",
                    "    for liquid in liquid_points:\n",
                    "        ax.scatter(liquid[1], liquid[0], label=liquid[2])\n",
                    "        plot_data.append(([liquid[1]], [liquid[0]], liquid[2]))\n",
                    "    \n",
                    "    if title:\n",
                    "        ax.set_title(title)\n",
                    "    ax.set_xlabel('Dispersive Component (mN/m)')\n",
                    "    ax.set_ylabel('Polar Component (mN/m)')\n",
                    "    ax.grid(True)\n",
                    "    ax.legend()\n",
                    "    \n",
                    "    # Determine axis limits with a margin\n",
                    "    if plot_data:\n",
                    "        x_max = max([max(x) for x, _, _ in plot_data if len(x) > 0]) * 1.1\n",
                    "        y_max = max([max(y) for _, y, _ in plot_data if len(y) > 0]) * 1.1\n",
                    "        ax.set_xlim([0, x_max])  # Ensure the X axis starts at 0 and has a margin\n",
                    "        ax.set_ylim([0, y_max])  # Ensure the Y axis starts at 0 and has a margin\n",
                    "    \n",
                    "    return plot_data\n",
                    "\n",
                    "# Initialize global variable to store liquid points\n",
                    "liquid_points = []"
                ]
            },
            {
                "cell_type": "code",
                "execution_count": None,
                "metadata": {},
                "source": [
                    "# Default values for materials and solvents\n",
                    "default_materials = [\n",
                    "    {'name': 'Material 1', 'polar': 5.43, 'dispersive': 40.0},\n",
                    "    {'name': 'Material 2', 'polar': 0.23, 'dispersive': 48.43}\n",
                    "]\n",
                    "\n",
                    "default_solvents = [\n",
                    "    {'name': 'Solvent 1', 'polar': 18.0, 'dispersive': 29.0},\n",
                    "    {'name': 'Solvent 2', 'polar': 2.3, 'dispersive': 25.5}\n",
                    "]"
                ]
            },
            {
                "cell_type": "code",
                "execution_count": None,
                "metadata": {},
                "source": [
                    "# Create widgets for the interactive interface\n",
                    "# Material input widgets\n",
                    "material_name = widgets.Text(value='Material 1', description='Name:')\n",
                    "material_polar = widgets.FloatText(value=5.43, description='Polar (mN/m):')\n",
                    "material_dispersive = widgets.FloatText(value=40.0, description='Dispersive (mN/m):')\n",
                    "material_theta = widgets.FloatSlider(value=0.0, min=0.0, max=180.0, step=1.0, description='Contact Angle (°):')\n",
                    "\n",
                    "# Solvent input widgets\n",
                    "solvent_name = widgets.Text(value='Solvent 1', description='Name:')\n",
                    "solvent_polar = widgets.FloatText(value=18.0, description='Polar (mN/m):')\n",
                    "solvent_dispersive = widgets.FloatText(value=29.0, description='Dispersive (mN/m):')\n",
                    "\n",
                    "# Plot settings widgets\n",
                    "plot_title = widgets.Text(value='Wetting Envelope', description='Title:')\n",
                    "correction_exponent = widgets.FloatSlider(value=2.0, min=1.0, max=2.0, step=0.01, description='Correction Exp:')\n",
                    "\n",
                    "# Create buttons\n",
                    "add_material_button = widgets.Button(description='Add Material')\n",
                    "add_solvent_button = widgets.Button(description='Add Solvent')\n",
                    "clear_button = widgets.Button(description='Clear All')\n",
                    "plot_button = widgets.Button(description='Plot')\n",
                    "\n",
                    "# Output area for the plot\n",
                    "output = widgets.Output()\n",
                    "\n",
                    "# Lists to store materials and solvents\n",
                    "materials = []\n",
                    "solvents = []\n",
                    "\n",
                    "# Add default materials and solvents\n",
                    "for material in default_materials:\n",
                    "    materials.append({\n",
                    "        'name': material['name'],\n",
                    "        'polar': material['polar'],\n",
                    "        'dispersive': material['dispersive'],\n",
                    "        'theta': 0.0\n",
                    "    })\n",
                    "\n",
                    "for solvent in default_solvents:\n",
                    "    solvents.append({\n",
                    "        'name': solvent['name'],\n",
                    "        'polar': solvent['polar'],\n",
                    "        'dispersive': solvent['dispersive']\n",
                    "    })\n",
                    "\n",
                    "# Display areas for materials and solvents\n",
                    "materials_output = widgets.Output()\n",
                    "solvents_output = widgets.Output()"
                ]
            },
            {
                "cell_type": "code",
                "execution_count": None,
                "metadata": {},
                "source": [
                    "# Define button callbacks\n",
                    "def add_material(b):\n",
                    "    materials.append({\n",
                    "        'name': material_name.value,\n",
                    "        'polar': material_polar.value,\n",
                    "        'dispersive': material_dispersive.value,\n",
                    "        'theta': material_theta.value\n",
                    "    })\n",
                    "    update_materials_display()\n",
                    "\n",
                    "def add_solvent(b):\n",
                    "    solvents.append({\n",
                    "        'name': solvent_name.value,\n",
                    "        'polar': solvent_polar.value,\n",
                    "        'dispersive': solvent_dispersive.value\n",
                    "    })\n",
                    "    update_solvents_display()\n",
                    "    \n",
                    "def clear_all(b):\n",
                    "    global materials, solvents, liquid_points\n",
                    "    materials = []\n",
                    "    solvents = []\n",
                    "    liquid_points = []\n",
                    "    update_materials_display()\n",
                    "    update_solvents_display()\n",
                    "    with output:\n",
                    "        clear_output()\n",
                    "\n",
                    "def plot_envelopes(b):\n",
                    "    global liquid_points\n",
                    "    with output:\n",
                    "        clear_output()\n",
                    "        \n",
                    "        # Prepare data for plotting\n",
                    "        components = [(str(m['polar']), str(m['dispersive'])) for m in materials]\n",
                    "        thetas = [str(m['theta']) for m in materials]\n",
                    "        legends = [m['name'] for m in materials]\n",
                    "        \n",
                    "        # Add solvents to liquid_points\n",
                    "        liquid_points = [(s['polar'], s['dispersive'], s['name']) for s in solvents]\n",
                    "        \n",
                    "        # Create figure and plot\n",
                    "        fig, ax = plt.subplots(figsize=(10, 8))\n",
                    "        plot_wetting_envelope(components, thetas, legends, ax, plot_title.value, correction_exponent.value)\n",
                    "        plt.tight_layout()\n",
                    "        plt.show()\n",
                    "\n",
                    "# Update display functions\n",
                    "def update_materials_display():\n",
                    "    with materials_output:\n",
                    "        clear_output()\n",
                    "        if materials:\n",
                    "            print(\"Added Materials:\")\n",
                    "            for i, m in enumerate(materials):\n",
                    "                print(f\"{i+1}. {m['name']}: Polar={m['polar']}, Dispersive={m['dispersive']}, θ={m['theta']}°\")\n",
                    "        else:\n",
                    "            print(\"No materials added.\")\n",
                    "\n",
                    "def update_solvents_display():\n",
                    "    with solvents_output:\n",
                    "        clear_output()\n",
                    "        if solvents:\n",
                    "            print(\"Added Solvents:\")\n",
                    "            for i, s in enumerate(solvents):\n",
                    "                print(f\"{i+1}. {s['name']}: Polar={s['polar']}, Dispersive={s['dispersive']}\")\n",
                    "        else:\n",
                    "            print(\"No solvents added.\")\n",
                    "\n",
                    "# Connect callbacks to buttons\n",
                    "add_material_button.on_click(add_material)\n",
                    "add_solvent_button.on_click(add_solvent)\n",
                    "clear_button.on_click(clear_all)\n",
                    "plot_button.on_click(plot_envelopes)\n",
                    "\n",
                    "# Initial display update\n",
                    "update_materials_display()\n",
                    "update_solvents_display()"
                ]
            },
            {
                "cell_type": "code",
                "execution_count": None,
                "metadata": {},
                "source": [
                    "# Create the layout for the interface\n",
                    "material_box = widgets.VBox([\n",
                    "    widgets.HTML(value=\"<h3>Material Properties</h3>\"),\n",
                    "    material_name,\n",
                    "    material_polar,\n",
                    "    material_dispersive,\n",
                    "    material_theta,\n",
                    "    add_material_button,\n",
                    "    materials_output\n",
                    "])\n",
                    "\n",
                    "solvent_box = widgets.VBox([\n",
                    "    widgets.HTML(value=\"<h3>Solvent Properties</h3>\"),\n",
                    "    solvent_name,\n",
                    "    solvent_polar,\n",
                    "    solvent_dispersive,\n",
                    "    add_solvent_button,\n",
                    "    solvents_output\n",
                    "])\n",
                    "\n",
                    "plot_settings_box = widgets.VBox([\n",
                    "    widgets.HTML(value=\"<h3>Plot Settings</h3>\"),\n",
                    "    plot_title,\n",
                    "    correction_exponent,\n",
                    "    widgets.HBox([plot_button, clear_button])\n",
                    "])\n",
                    "\n",
                    "# Arrange the interface\n",
                    "top_row = widgets.HBox([material_box, solvent_box])\n",
                    "app = widgets.VBox([top_row, plot_settings_box, output])\n",
                    "\n",
                    "# Display the interface\n",
                    "display(app)"
                ]
            },
            {
                "cell_type": "markdown",
                "metadata": {},
                "source": [
                    "## How to Use This Applet\n",
                    "\n",
                    "1. **Materials**: Enter the properties of materials (polar and dispersive components) and their contact angles.\n",
                    "2. **Solvents**: Enter the properties of solvents (polar and dispersive components).\n",
                    "3. **Plot Settings**: Adjust the plot title and correction exponent as needed.\n",
                    "4. **Buttons**:\n",
                    "   - **Add Material**: Add a material with the specified properties.\n",
                    "   - **Add Solvent**: Add a solvent with the specified properties.\n",
                    "   - **Plot**: Generate the wetting envelope plot.\n",
                    "   - **Clear All**: Remove all materials and solvents.\n",
                    "\n",
                    "The plot shows wetting envelopes for each material and points for each solvent. If a solvent point falls inside a material's wetting envelope, it indicates that the solvent will wet the material."
                ]
            },
            {
                "cell_type": "code",
                "execution_count": None,
                "metadata": {},
                "source": [
                    "# Add Voila configuration for deployment\n",
                    "# This cell is not necessary for running in Jupyter, but helps with Voila deployment\n",
                    "from IPython.display import HTML\n",
                    "HTML('''\n",
                    "<script>\n",
                    "  function code_toggle() {\n",
                    "    if (code_shown){\n",
                    "      $('div.input').hide('500');\n",
                    "      $('#toggleButton').val('Show Code')\n",
                    "    } else {\n",
                    "      $('div.input').show('500');\n",
                    "      $('#toggleButton').val('Hide Code')\n",
                    "    }\n",
                    "    code_shown = !code_shown\n",
                    "  }\n",
                    "  \n",
                    "  $( document ).ready(function(){\n",
                    "    code_shown=false;\n",
                    "    $('div.input').hide();\n",
                    "  });\n",
                    "</script>\n",
                    "<form action=\"javascript:code_toggle()\">\n",
                    "<input type=\"submit\" id=\"toggleButton\" value=\"Show Code\">\n",
                    "</form>\n",
                    "''')"
                ]
            }
        ],
        "metadata": {
            "kernelspec": {
                "display_name": "Python 3",
                "language": "python",
                "name": "python3"
            },
            "language_info": {
                "codemirror_mode": {
                    "name": "ipython",
                    "version": 3
                },
                "file_extension": ".py",
                "mimetype": "text/x-python",
                "name": "python",
                "nbconvert_exporter": "python",
                "pygments_lexer": "ipython3",
                "version": "3.8.10"
            }
        },
        "nbformat": 4,
        "nbformat_minor": 4
    }
    
    import json
    f.write(json.dumps(notebook_content))

# Create a requirements.txt file for the project
with open('requirements.txt', 'w') as f:
    f.write(
        '''
        numpy>=1.19.0
        matplotlib>=3.3.0
        ipywidgets>=7.6.0
        voila>=0.2.0
        jupyter>=1.0.0
''')

# Create a README.md file with instructions
with open('README.md', 'w') as f:
    f.write('''# Wetting Envelope Interactive Applet  
    This project provides an interactive applet for plotting wetting envelopes for materials and solvents based on their surface energy components.  
    
    ## Requirements  
    Install the required packages:  
    pip install -r requirements.txt  
    ''')